# Gestion des listes de signaux stockés au format HDF5

L'OPSET contient essentiellement une classe `Opset` permettant de manipuler et d'accéder facilement à des signaux stockés dans un fichier HDF5.

Chaque signal est un pandas.DataFrame `df` dont le nom est tocké dans `df.index.name`.
Ces noms sont indépendants des noms d'enregistrement dans le fichier mais il est préférable de conserver les mêmes.

Il est préférable d'avoir stocké ainsi des signaux ayant les mêmes noms de colonnes. Les unités de chaque colonne si elles existent sont stockées entre crochets après le nom de la variable.

Une telle liste sera appelée "liste d'opérations" ou OPSET.

In [110]:
from importlib import reload
import numpy as np
import pandas as pd
import tabata.opset as op

Je mets souvent une commande `reload` pour vérifier la mise au point de mes codes.

In [111]:
reload(op)

<module 'tabata.opset' from 'H:\\wrk\\_jlpath\\tabata\\opset.py'>

## 1/ Visualisation des données.

La classe `Opset` permet d'accéder aux signaux, la méthode `plot()` offre une interface graphique sympatique pour naviguer au sein de la liste.

In [112]:
storename = "data/in/AFL1EB.h5"
ds = op.Opset(storename)

In [113]:
ds.plot()

In [114]:
ds

OPSET 'data/in/AFL1EB.h5' de 52 signaux.
        position courante : sigpos  = 0
        variable courante : colname = ALT[m]
        phase surlignée   : phase   = None

Les données de ce jeu d'exemple représentent des mesures simulées de vols d'un avion.
    
* ALT[m] : l'altitude de l'avion en mètres.
* Tisa[K] : la température standard en Kelvin.
* TAS[m/s] : La vitesse air de l'avion (Total Air Speed) en mètre par seconde.
* Vz[m/s] : la vitesse de montée en mètre par seconde.
* Masse[kg] : la masse de l'avion en kilogramme.
* F[N] : la poussée en Newton.

Remarquez que 
* les enregistrements 6, 7, 8 et 9 sont vides, en fait ce n'est que du bruit de capteur ;
* les enregistrements 45 et 50 ont un problème d'horodatatage ;
* l'enregistrement 10 présente un atterrissage raté.

L'enregistrement courant peut être récupéré facilement.

In [115]:
ds.df

,ALT[m],Tisa[K],TAS[m/s],Vz[m/s],Masse[kg],F[N]
record_00,,,,,,
2012-07-10 11:08:00,-1.267968,296.391008,0.0,0.017541,15217.558677,0.0
2012-07-10 11:08:01,-1.267968,296.391008,0.0,0.016004,15217.558677,0.0
2012-07-10 11:08:02,-1.267968,296.391008,0.0,0.014466,15217.558677,0.0
2012-07-10 11:08:03,-1.267968,296.391008,0.0,0.012929,15217.558677,0.0
2012-07-10 11:08:04,-1.267968,296.391008,0.0,0.011392,15217.558677,0.0
...,...,...,...,...,...,...
2012-07-10 13:11:44,49.450752,296.064959,0.0,-0.023152,13792.505334,0.0
2012-07-10 13:11:45,49.450752,296.064959,0.0,-0.026291,13792.505334,0.0
2012-07-10 13:11:46,49.450752,296.064959,0.0,-0.029430,13792.505334,0.0


Notez que le nom de l'enregistrement est stocké dans le nom de l'index du DataFrame. C'est une propriété qui est persistante quand on sauvegarde le DataFrame.

On peut directement demander l'affichage spécifique d'un des signaux.

In [116]:
ds.plot(pos=21,name="Vz[m/s]")

Cette demande peut aussi se faire dès l'instanciation.

In [117]:
ds = op.Opset(storename,pos=5,name="TAS[m/s]")
ds.plot()

## 2/ Itérations
Il est possible d'itérer sur la liste de signaux.

L'itération met à jour le signal courant du sélecteur.

In [118]:
n=0
for df in ds.iterator(3):
    t0 = df.index[0]
    print("{:2d} : {:10s} --> {}".format(n,df.index.name,t0))
    n = n+1
    
print('Boucle n°0 :',ds.df.index.name)

n=0
for df in ds.iterator(3,5):
    t0 = df.index[0]
    print("{:2d} : {:10s} --> {}".format(n,df.index.name,t0))
    n = n+1
    
print('Boucle n°1 :',ds.df.index.name)

n=0
for df in ds.iterator(3,9):
    t0 = df.index[0]
    print("{:2d} : {:10s}  --> {}".format(n,df.index.name,t0))
    n = n+1
    
print('Boucle n°2 :',ds.df.index.name)

n=0
for df in ds.iterator([1, 11, 23]):
    t0 = df.index[0]
    print("{:2d} : {:10s}  --> {}".format(n,df.index.name,t0))
    n = n+1
    
print('Boucle n°3 :',ds.df.index.name)

 0 : record_00  --> 2012-07-10 11:08:00
 1 : record_01  --> 2012-07-10 14:10:00
 2 : record_02  --> 2012-07-27 03:40:00
Boucle n°0 : record_02
 0 : record_03  --> 2012-07-27 06:34:00
 1 : record_04  --> 2012-08-03 01:24:00
Boucle n°1 : record_04
 0 : record_03   --> 2012-07-27 06:34:00
 1 : record_04   --> 2012-08-03 01:24:00
 2 : record_05   --> 2012-08-03 03:25:00
 3 : record_06   --> 2012-12-28 09:31:00
 4 : record_07   --> 2013-01-05 06:26:00
 5 : record_08   --> 2013-01-05 09:43:00
Boucle n°2 : record_08
 0 : record_01   --> 2012-07-10 14:10:00
 1 : record_11   --> 2013-01-19 20:25:00
 2 : record_23   --> 2013-02-07 14:02:00
Boucle n°3 : record_23


## 3/ Création d'un nouvel Opset
À partir d'une première liste d'opérations il est possible d'en créer une nouvelle assez facilement à l'aide de l'objet `Opset`.

Dans notre exemple on a vu qu'il existe des vols mal enregistrés, on va les supprimer, par ailleurs il y a un problème de changement de date qui a abimé l'index temporel (à la seconde ici) ce que l'on va corriger en créant un jeu de données propres.

La méthode `put(df,record)` permet une modification rapide d'un enregistrement. Par défaut, si aucun nom d'enregistrement n'est donné en second paramètre, on regarde si `df.index.name` existe. De façon équivalente, si l'index n'est pas nommé mais qu'un nom d'enregistrement sera passé, la fonction `put(df,record)` rajoutera le nom d'index. 

La méthode `clean()` permet d'effacer le fichier avant de recommencer une écriture.

In [119]:
# Construction d'un hdf5set sans données anormales.
cleanstore = "data/out/AFL1EB_C.h5"
dsc = op.Opset(cleanstore)
dsc.clean() # Au cas où le fichier existerait déjà.

for df in ds.iterator():
    if max(df["F[N]"])>0: # Les données ont été bien enregistrées.
        x = df.index
        t = (x-x[0]).total_seconds()
        dt = np.diff(t)
        i = np.argwhere(dt != dt[1]) # on détecte un problème.
        if len(i)>0:
            name = df.index.name
            df.index = pd.date_range(x[0],periods=len(df),freq=x[1]-x[0]) # détruit le nom
            df.index.name = name # Ne pas oublier de récupérer le nom.
            dsc.put(df) # Ici on aurait pu mettre name.

La fonction `current_record()` est un racourci pour obtenir le nom de l'enregistrement courant.
Dans une itération, vu que l'enregistrement courant est celui qui est renvoyé, si on ne précise pas de nouveau nom d'enregistrement, c'est l'enregistrement courant qui est modifié par `put()` dans l'exemple donné.

In [120]:
for df in dsc.iterator(4):
    print(dsc.current_record(), ':', df.index.name)

/record_00 : record_00
/record_01 : record_01
/record_02 : record_02
/record_03 : record_03


La méthode `rewind()` est un raccourci pour remettre le pointeur au début du fichier.

In [121]:
dsc.rewind()
print(dsc.current_record(), ':', dsc.df.index.name)

/record_00 : record_00


### Modification de l'Opset

Une fonction intéressante de la méthode `plot()` est la possibilité d'afficher en surimpression une partie du signal. Pour cela il suffit de rajouter une variable booléenne aux signaux qui indique quelles partie du signal identifier.

Dans cet exemple nous essayons d'extraire la croisière.

In [122]:
for df in dsc.iterator():
    mx = max(df["ALT[m]"])
    df["CR"] = (df["ALT[m]"]>mx-2000) & (abs(df["Vz[m/s]"])<1)
    dsc.put(df)

In [123]:
dsc.rewind().plot("CR")

In [124]:
dsc

OPSET 'data/out/AFL1EB_C.h5' de 48 signaux.
        position courante : sigpos  = 0
        variable courante : colname = ALT[m]
        phase surlignée   : phase   = CR

### Quelques éléments techniques de la méthode `plot()`.

#### Gestion des signaux multivariés
La classe `Opset` permet de gérer des données stockées sous la forme d'une liste d'observations temporelles. Chaque observation est un signal multivarié indexé par le temps. Cette liste doit être stockée dans un fichier au format HDF5 où chaque enregistrement est une observations.

* Les observations sont lues dans l'ordre initial de stockage par un itérateur créé par la méthode `iterator()`.
* Chaque signal (observation) est synchrones : toutes les colonnes ont la même longuer et le signal est stocké sous la forme d'un DataFrame pandas (`df`).
* Le nom du signal est stocké dans le nom de l'index (`df.index.name`), ce qui permet de nommer les enregistrements indépendamment de leur nom. (Il se trouve que ce nom est persistant à la sauvegarde HDF5.)
* Le nom de chaque colonne correspond au nom de la variable suivi entre crochets de son unité.

#### Affichage des données.
La méthode `plot()` sert à afficher les données de manière interactive à l'aide de Plotly au sein d'un notebook Python. Pour cela l'import du package hdf5set exécute la fonction `init_notebook_mode` de Plotly permettant d'interagir au sein du notebook à l'aide de la fonction `iplot()` ou directement par le renderer de base de Jupyter.

Pour naviguer entre les signaux, l'Opset utilise des gadgets (boutons, scrollbar, menu) pour passer d'un enregistrement à un autre. Ces gadgets sont issus du packae ipywidgets qui est compatible avec Plotly. Le code de la méthode `plot()` est un exemple assez parlant de l'exploitation de gadgets interactifs au sein d'un notebook Jupyter.

* Un menu permet de sélectionner une colonne à afficher.
* Deux boutons (Previous et Next) passent d'un signal au suivant dans l'ordre des enregistrements.
* Une scrollbar verticale sur la droite rappelle l'enregistrement en cours de visualisation et permet aussi de se déplacer aléatoirement entre les enregistrements.

##### Point technique
La méthode `plot()` appelle une méthode intermédiaire : `make_figure()` qui génère les composants de la figure et la fonction d'interactivité du callback. Cette technique permettra de facilité la dérivation de classes spécifiques et la possibilité de reconstruire des représentations graphiques adaptées (voir Instants).

#### Variables locales
La liste ordonnée des enregistrements est sauvegardée dans une variables `.records`, le signal en cours d'affichage est stocké dans `.sigpos` (en commençant par le n°1), et le nom de la variable à afficher est conservé dans `.colname`.

#### Fonctionnement de l'interactivité
L'interactivité est donnée par l'appel

    out = widgets.interactive(update_plot, colname=wd, sigpos=ws)
  
`wd` est un pointeur vers le menu Dropdown contenat la liste des variables du signal en cours d'affichage et `ws` un pointeur vers la scrollbar.

       wd = widgets.Dropdown(options=df.columns, 
                             description="Variable :")
       ws = widgets.IntSlider(value=1, min=1, max=nbmax, step=-1,
                              orientation='vertical',
                              description='Record',
                              layout=widgets.Layout(height='400px'))
                               
Cette dernière est en mode 'vertical' et sa hauteur est fixée à 400 pixels. Le nombre maximum d'enregistrements a été stocké dans la variable `nbmax`. La fonction `interactive` lie le contenu des gadgets au variables locales `colanme` et `sigpos`qui sont passées à la fonction locale `update_plot()` qui fait le travail d'affichage en mettant à jour les éléments définis par le premier affichage initila à l'apple de `.plot()`.

Si la scrollbar apparait sur la droite de l'affichage c'est parce que l'on a juxtaposé deux HBox dans une VBox que l'on revoie comme retour de `.plot()` :

        boxes = widgets.VBox([widgets.HBox([wd, wbp, wbn]), 
                              widgets.HBox([f, ws])])
        return boxes

La première HBox contient le menu et les deux boutons, et juste en dessous nous plaçons une seconde Hbox qui contient d'abbord la dfigure `f` créée initialement et à sa droite la scrollbar verticale.

    f = go.FigureWidget(data, layout)
    
L'interaction des boutons est assez simple : on le lie à une fonction callback locale `wb_on_click()` que l'on associe à chaque widget par l'appel de `.on_click()`. En fait cette fonction se contente de modifier la valeur de la scrollbar ce qui entrainera automatiquement par interactivité une modification de l'afficahe comme si on avait directement cliqué sur la scrollbar.

#### Mise en évidence d'une partie des données
Le Selector bénéficie de deux fonction d'affichage supplémentaire. La première consiste à mettre en évidence par un coloriage en rouge une partie du signal.
Pour cela il faut qu'une variable booléenne soit présente dans le signal multivarié comme une colonne particulière. Le nom de cette variable peut être passé en argument `phase`de `plot(phase)` et les points 'True' de cette 'phase' seront affichés en rouge.

## 4/ Affichage simplifié des DataFrames
Deux fonction simplifient l'affichage d'un opset.

* `selplot(df,[variable])`affiche un DataFrame variable par variable.
* `byunitplot(df,[yunit],[xunit],[title])` affiche le DataFrame en sélectionnant les variables, unité par unité sur le même graphe.

In [125]:
df = dsc.df

Affichage des courbes une à une.
On peut préciser une partie du nom de la variable.

In [126]:
op.selplot(df,"V")

interactive(children=(Dropdown(description='Variable :', index=3, options=('ALT[m]', 'Tisa[K]', 'TAS[m/s]', 'V…

Affichage des courbes par unité.

In [127]:
op.byunitplot(df,'m/s')

_Jérôme Lacaille (YOR)_